In [2]:
import pandas as pd
from shapely import wkt
import json

In [4]:
# Load the Excel file
file_path = '/Users/fer/pCloud Drive/2_STORED/0_GEODATABASE_tables/2_DRONE_table.xlsx'
data = pd.read_excel(file_path)

# Select relevant columns for conversion to JSON
relevant_columns = ['dwc:eventID', 'dwc:eventDate', 'dwc:decimalLatitude', 'dwc:decimalLongitude', 'dwc:institutionCode', 'dwc:footprintWKT','GSD', 'contact','url']

# Ensure the columns are in the correct format for Leaflet
data_filtered = data[relevant_columns].copy()

# Rename columns to match the JSON structure
data_filtered.rename(columns={
    'dwc:eventID' : 'eventID', 
    'dwc:eventDate' : 'eventDate',
    'dwc:decimalLatitude': 'decimalLatitude',
    'dwc:decimalLongitude': 'decimalLongitude',
    'dwc:institutionCode': 'institutionCode',
    'GSD': 'resolution'
}, inplace=True)


# Convert footprintWKT to list of coordinate pairs for polygons (Leaflet expects [latitude, longitude])
def convert_wkt_to_coordinates(wkt_str):
    geom = wkt.loads(wkt_str)
    return [(coord[1], coord[0]) for coord in geom.exterior.coords]

data_filtered['polygon'] = data_filtered['dwc:footprintWKT'].apply(convert_wkt_to_coordinates)

# Drop the original footprintWKT column
data_filtered.drop(columns=['dwc:footprintWKT'], inplace=True)

# Convert the DataFrame to a JSON-like structure
json_data = data_filtered.to_dict(orient='records')

# Prepare the JavaScript variable with data
js_variable = f"var drone = {json.dumps(json_data, indent=4)};"

# Save to a .js file
output_js_path = '/Users/fer/Documents/PYTHON/15_GEAM_GDB_leaflet/docs/data/2_DRONE_table.js'
with open(output_js_path, 'w') as js_file:
    js_file.write(js_variable)


# Print the path to the JSON file
print(f"JSON file saved to: {output_js_path}")

JSON file saved to: /Users/fer/Documents/PYTHON/15_GEAM_GDB_leaflet/docs/data/2_DRONE_table.js
